In [1]:
import brightway2 as bw
import bw2data as bd
import bw2analyzer as ba

#reduce?
import ast
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dopo
from dopo import*

In [2]:
bd.projects.set_current("premise-validation-try1")
bw.bw2setup()

bio3=bw.Database('biosphere3')
ei39=bw.Database('ecoinvent 3.9.1 cutoff')
ei39SSP2=bw.Database('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27')

Biosphere database already present!!! No setup is needed


FILTERS

In [3]:
#sector filters file names/paths

cement = 'cement_small.yaml'
electricity = 'electricity_small.yaml'
fuels= 'fuels_small.yaml'
steel = 'steel_small.yaml'
transport = 'transport_small.yaml'

In [4]:
files_dict={}
files_dict['Cement']={'yaml': 'yamls\cement_small.yaml', 
                      'yaml identifier': 'Cement'}
files_dict['Steel']= {'yaml':'yamls\steel_small.yaml',
                            'yaml identifier': 'Steel'} #yaml identifier is the name of the filter in the yaml file, in the first line.
files_dict

{'Cement': {'yaml': 'yamls\\cement_small.yaml', 'yaml identifier': 'Cement'},
 'Steel': {'yaml': 'yamls\\steel_small.yaml', 'yaml identifier': 'Steel'}}

In [5]:
import dopo.filter_sectors

#for plot 1 and 2
dictionary_one = dopo.filter_sectors.process_yaml_files(files_dict, ei39SSP2)

#for comparison
premise_dict = dopo.filter_sectors.process_yaml_files(files_dict, ei39SSP2)
ecoinvent_dict = dopo.filter_sectors.process_yaml_files(files_dict, ei39)


Processing Cement with database ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27
Activities for Cement:
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '86841f8c7ee2668f244d3b8e34f41932')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'df49e8f525497f2fbd56bcdc80ff0cde')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '3c16b45db40210cd97de6574b2f47aaf')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'fcb666edf2a01467e555eeff5b4a5bbb')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'f8b84f45f50d3bd7ff4feaabdb493f6a')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'a3c2064d83411f7963af550c04c869a1')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '36a53c174f34e672bc15b7e55563685e')
Processing Steel with database ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27
Activities for Steel:
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'af6bd1221fc0206541fbaf481397bf0d')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '2baa0deb3adc89dfe8cb89d5e078ba8d

In [10]:
finder=dopo.methods.MethodFinder()

finder.find_and_create_method(criteria=['IPCC', '2013', 'GWP100'], exclude=['no LT'])
finder.find_and_create_method(criteria=['EN15804','Cumulative', 'non-renewable' ])
#finder.find_and_create_method(criteria=['land occupation','selected'])
#finder.find_and_create_method(criteria=['EN15804','fresh water'])
method_dict=finder.get_all_methods()
method_dict

{'method_1': {'object': Brightway2 Method: IPCC 2013: climate change: global warming potential (GWP100),
  'method name': ('IPCC 2013',
   'climate change',
   'global warming potential (GWP100)'),
  'short name': 'global warming potential (GWP100)',
  'unit': 'kg CO2-Eq'},
 'method_2': {'object': Brightway2 Method: EN15804: inventory indicators ISO21930: Cumulative Energy Demand - non-renewable energy resources,
  'method name': ('EN15804',
   'inventory indicators ISO21930',
   'Cumulative Energy Demand - non-renewable energy resources'),
  'short name': 'Cumulative Energy Demand - non-renewable energy resources',
  'unit': 'megajoule'}}

LCA Tables

In [26]:
import math
import operator
from os.path import commonprefix

import bw2calc as bc
import bw2data as bd
import numpy as np
import pandas as pd
import tabulate
from pandas import DataFrame


def aggregated_dict(activity):
    """Return dictionary of inputs aggregated by input reference product."""
    results = {}
    for exc in activity.technosphere():
        results[exc.input["reference product"]] = (
            results.setdefault(exc.input["reference product"], 0) + exc["amount"]
        )

    for exc in activity.biosphere():
        results[exc.input["name"]] = (
            results.setdefault(exc.input["name"], 0) + exc["amount"]
        )

    return results



def compare_dictionaries(one, two, rel_tol=1e-4, abs_tol=1e-9):
    """Compare two dictionaries with form ``{str: float}``, and return a set of keys where differences where present.

    Tolerance values are inputs to `math.isclose <https://docs.python.org/3/library/math.html#math.isclose>`__."""
    return (
        set(one)
        .symmetric_difference(set(two))
        .union(
            {
                key
                for key in one
                if key in two
                and not math.isclose(
                    a=one[key], b=two[key], rel_tol=rel_tol, abs_tol=abs_tol
                )
            }
        )
    )




def find_differences_in_inputs(
    activity, rel_tol=1e-4, abs_tol=1e-9, locations=None, as_dataframe=False
):
    """Given an ``Activity``, try to see if other activities in the same database (with the same name and
    reference product) have the same input levels.

    Tolerance values are inputs to `math.isclose <https://docs.python.org/3/library/math.html#math.isclose>`__.

    If differences are present, a difference dictionary is constructed, with the form:

    .. code-block:: python

        {Activity instance: [(name of input flow (str), amount)]}

    Note that this doesn't reference a specific exchange, but rather sums **all exchanges with the same input reference product**.

    Assumes that all similar activities produce the same amount of reference product.

    ``(x, y)``, where ``x`` is the number of similar activities, and ``y`` is a dictionary of the differences. This dictionary is empty if no differences are found.

    Args:
        activity: ``Activity``. Activity to analyze.
        rel_tol: float. Relative tolerance to decide if two inputs are the same. See above.
        abs_tol: float. Absolute tolerance to decide if two inputs are the same. See above.
        locations: list, optional. Locations to restrict comparison to, if present.
        as_dataframe: bool. Return results as pandas DataFrame.

    Returns:
        dict or ``pandas.DataFrame``.


    """
    assert isinstance(activity, bd.backends.proxies.Activity)

    try:
        similar = [
            obj
            for obj in bd.Database(activity["database"])
            if obj != activity
            and obj.get("reference product") == activity.get("reference product")
            and obj.get("name") == activity["name"]
            and (not locations or obj.get("location") in locations)
        ]
    except KeyError:
        raise ValueError("Given activity has no `name`; can't find similar names")

    result = {}

    origin_dict = aggregated_dict(activity)

    for target in similar:
        target_dict = aggregated_dict(target)
        difference = compare_dictionaries(origin_dict, target_dict, rel_tol, abs_tol)
        if difference:
            if activity not in result:
                result[activity] = {}
            result[activity].update(
                {key: value for key, value in origin_dict.items() if key in difference}
            )
            result[target] = {
                key: value for key, value in target_dict.items() if key in difference
            }

    if as_dataframe:
        df = DataFrame(
            [{"location": obj.get("location"), **result[obj]} for obj in result]
        )
        df.set_index("location", inplace=True)
        return df
    else:
        return result




def compare_activities_by_lcia_score(activities, lcia_method, band=0.1):
    """Compare selected activities to see if they are substantially different.

    Substantially different means that all LCIA scores lie within a band of ``band * max_lcia_score``.

    Inputs:

        ``activities``: List of ``Activity`` objects.
        ``lcia_method``: Tuple identifying a ``Method``

    Returns:

        Nothing, but prints to stdout.

    """
    import bw2calc as bc

    activities = [bd.get_activity(obj) for obj in activities]

    lca = bc.LCA({a: 1 for a in activities}, lcia_method)
    lca.lci()
    lca.lcia()

    # First pass: Are all scores close?
    scores = []

    for a in activities:
        lca.redo_lcia({a.key: 1})
        scores.append(lca.score)

    if abs(max(scores) - min(scores)) < band * abs(max(scores)):
        print("All activities similar")
        return
    else:
        print("Differences observed. LCA scores:")
        for x, y in zip(scores, activities):
            print("\t{:5.3f} -> {}".format(x, y.key))




def find_leaves(
    activity,
    lcia_method,
    results=None,
    lca_obj=None,
    amount=1,
    total_score=None,
    level=0,
    max_level=3,
    cutoff=2.5e-2,
):
    """Traverse the supply chain of an activity to find leaves - places where the impact of that
    component falls below a threshold value.

    Returns a list of ``(impact of this activity, amount consumed, Activity instance)`` tuples."""
    first_level = results is None

    activity = bd.get_activity(activity)

    if first_level:
        level = 0
        results = []

        lca_obj = bc.LCA({activity: amount}, lcia_method)
        lca_obj.lci()
        lca_obj.lcia()
        total_score = lca_obj.score
    else:
        lca_obj.redo_lcia({activity.key: amount})

        # If this is a leaf, add the leaf and return
        if abs(lca_obj.score) <= abs(total_score * cutoff) or level >= max_level:

            # Only add leaves with scores that matter
            if abs(lca_obj.score) > abs(total_score * 1e-4):
                results.append((lca_obj.score, amount, activity))
            return results

        else:
            # Add direct emissions from this demand
            direct = (
                lca_obj.characterization_matrix
                * lca_obj.biosphere_matrix
                * lca_obj.demand_array
            ).sum()
            if abs(direct) >= abs(total_score * 1e-4):
                results.append((direct, amount, activity))

    for exc in activity.technosphere():
        find_leaves(
            activity=exc.input,
            lcia_method=lcia_method,
            results=results,
            lca_obj=lca_obj,
            amount=amount * exc["amount"],
            total_score=total_score,
            level=level + 1,
            max_level=max_level,
            cutoff=cutoff,
        )

    return sorted(results, reverse=True)



def get_cpc(activity):
    try:
        return next(
            cl[1] for cl in activity.get("classifications", []) if cl[0] == "CPC"
        )
    except StopIteration:
        return



def get_value_for_cpc(lst, label):
    for elem in lst:
        if elem[2] == label:
            return elem[0]
    return 0



def group_leaves(leaves):
    """Group elements in ``leaves`` by their `CPC (Central Product Classification) <https://unstats.un.org/unsd/classifications/Econ/cpc>`__ code.

    Returns a list of ``(fraction of total impact, specific impact, amount, Activity instance)`` tuples."""
    results = {}

    for leaf in leaves:
        cpc = get_cpc(leaf[2])
        if cpc not in results:
            results[cpc] = np.zeros((2,))
        results[cpc] += np.array(leaf[:2])

    return sorted([v.tolist() + [k] for k, v in results.items()], reverse=True)



def compare_activities_by_grouped_leaves(
    activities,
    lcia_method,
    mode="relative",
    max_level=4,
    cutoff=0.2,
    output_format="list",
    str_length=50,
):
    """Compare activities by the impact of their different inputs, aggregated by the product classification of those inputs.

    Args:
        activities: list of ``Activity`` instances.
        lcia_method: tuple. LCIA method to use when traversing supply chain graph.
        mode: str. If "relative" (default), results are returned as a fraction of total input. Otherwise, results are absolute impact per input exchange.
        max_level: int. Maximum level in supply chain to examine.
        cutoff: float. Fraction of total impact to cutoff supply chain graph traversal at.
        output_format: str. See below.
        str_length; int. If ``output_format`` is ``html``, this controls how many characters each column label can have.

    Raises:
        ValueError: ``activities`` is malformed.

    Returns:
        Depends on ``output_format``:

        * ``list``: Tuple of ``(column labels, data)``
        * ``html``: HTML string that will print nicely in Jupyter notebooks.
        * ``pandas``: a pandas ``DataFrame``.

    """
    for act in activities:
        if not isinstance(act, bd.backends.peewee.proxies.Activity):
            raise ValueError("`activities` must be an iterable of `Activity` instances")

    objs = [
        group_leaves(find_leaves(act, lcia_method, max_level=max_level, cutoff=cutoff))
        for act in activities
    ]
    sorted_keys = sorted(
        [
            (max([el[0] for obj in objs for el in obj if el[2] == key]), key)
            for key in {el[2] for obj in objs for el in obj}
        ],
        reverse=True,
    )
    name_common = commonprefix([act["name"] for act in activities])

    if " " not in name_common:
        name_common = ""
    else:
        last_space = len(name_common) - operator.indexOf(reversed(name_common), " ")
        name_common = name_common[:last_space]
        print("Omitting activity name common prefix: '{}'".format(name_common))

    product_common = commonprefix(
        [act.get("reference product", "") for act in activities]
    )

    lca = bc.LCA({act: 1 for act in activities}, lcia_method)
    lca.lci()
    lca.lcia()

    labels = [
        "activity",
        "product",
        "location",
        "unit",
        "total",
        "direct emissions",
    ] + [key for _, key in sorted_keys]
    data = []
    for act, lst in zip(activities, objs):
        lca.redo_lcia({act.key: 1})
        data.append(
            [
                act["name"].replace(name_common, ""),
                act.get("reference product", "").replace(product_common, ""),
                act.get("location", "")[:25],
                act.get("unit", ""),
                lca.score,
            ]
            + [
                (
                    lca.characterization_matrix
                    * lca.biosphere_matrix
                    * lca.demand_array
                ).sum()
            ]
            + [get_value_for_cpc(lst, key) for _, key in sorted_keys]
        )

    data.sort(key=lambda x: x[4], reverse=True)

    if mode == "relative":
        for row in data:
            for index, point in enumerate(row[5:]):
                row[index + 5] = point / row[4]

    if output_format == "list":
        return labels, data
    elif output_format == "pandas":
        return pd.DataFrame(data, columns=labels)
    elif output_format == "html":
        return tabulate.tabulate(
            data,
            [x[:str_length] for x in labels],
            tablefmt="html",
            floatfmt=".3f",
        )


In [18]:
def compare_activities_multiple_methods(
    activities_list, methods, identifier, output_format="pandas", mode="absolute"
):
    """
    Compares a set of activities by multiple methods, stores each generated dataframe as a variable (the method is the variable name) in a dictionary.

    :param activities_list: List of activities to compare
    :param methods: List of Brightway Method objects
    :param identifier: A string used in defining the variable names to better identify comparisons (e.g. sector name).
    :param output_format: Output format for the comparison (default: 'pandas')
    :param mode: Mode for the comparison (default: 'absolute'; others: 'relative')
    :return: Dictionary of resulting dataframes from the comparisons
    """
    dataframes_dict = {}

    for method_key, method_details in methods.items():
        result = compare_activities_by_grouped_leaves(
            activities_list,
            method_details["object"].name,
            output_format=output_format,
            mode=mode,
        )

        # Create a variable name using the method name tuple and identifier
        method_name = method_details["object"].name[2].replace(" ", "_").lower()
        var_name = f"{identifier}_{method_name}"

        # add two columns method and method unit to the df
        result["method"] = str(method_details["object"].name[2])
        result["method unit"] = str(method_details["object"].metadata["unit"])

        # order the columns after column unit
        cols = list(result.columns)
        unit_index = cols.index("unit")
        cols.insert(unit_index + 1, cols.pop(cols.index("method")))
        cols.insert(unit_index + 2, cols.pop(cols.index("method unit")))
        result = result[cols]

        # Order the rows based on 'activity' and 'location' columns
        result = result.sort_values(["activity", "location"])

        # Reset the index numbering
        result = result.reset_index(drop=True)

        # Store the result in the dictionary
        dataframes_dict[var_name] = result

    return dataframes_dict

In [19]:
def small_inputs_to_other_column(dataframes_dict, cutoff=0.01):
    """
    Aggregate values into a new 'other' column for those contributing less than or equal to the cutoff value to the 'total' column value.
    Set the aggregated values to zero in their original columns.
    Remove any columns that end up containing only zeros.

    :param dataframes_dict: the dictionary

    """

    processed_dict = {}

    for key, df in dataframes_dict.items():
        # Identify the 'total' column
        total_col_index = df.columns.get_loc("total")

        # Separate string and numeric columns
        string_cols = df.iloc[:, :total_col_index]
        numeric_cols = df.iloc[:, total_col_index:]
        numeric_cols = numeric_cols.astype(float)

        # Calculate the threshold for each row (1% of total)
        threshold = numeric_cols["total"] * cutoff

        # Create 'other' column
        numeric_cols["other"] = 0.0

        # Process each numeric column (except 'total' and 'other')
        for col in numeric_cols.columns[1:-1]:  # Skip 'total' and 'other'
            # Identify values less than the threshold
            mask = (
                abs(numeric_cols[col]) < threshold
            )  # abs() to include negative contributions

            # Add these values to 'other'
            numeric_cols.loc[mask, "other"] += numeric_cols.loc[mask, col]

            # Set these values to zero in the original column
            numeric_cols.loc[mask, col] = 0

        # Remove columns with all zeros (except 'total' and 'other')
        cols_to_keep = ["total"] + [
            col
            for col in numeric_cols.columns[1:-1]
            if not (numeric_cols[col] == 0).all()
        ]
        cols_to_keep.append("other")

        numeric_cols = numeric_cols[cols_to_keep]

        # Combine string and processed numeric columns
        processed_df = pd.concat([string_cols, numeric_cols], axis=1)

        # Sort columns by total
        processed_df = processed_df.sort_values("total", ascending=False)

        # Store the processed DataFrame in the result dictionary
        processed_dict[key] = processed_df

    return processed_dict

In [20]:
def sector_lca_scores(main_dict, method_dict):
    '''
    Generates the LCA score tables for activity list of each sector.
    The tables contain total scores and cpc input contributions.
    This is done by each method defined in the method dictionary.

    :param main_dict: dictionary which is returned by process_yaml_files function
    :param method_dict: dictionary which is created with MethodFinder class

    It returns the main dictionary updated as scores dictionary which also holds the former information for each sector.
    The LCA scores are stored by method name in the respective sector dictionary within the main dictionary.
    '''

    # Initialize scores_dict as a copy of main_dict
    scores_dict = main_dict.copy()

    # Loop through each sector in main_dict
    for sector in scores_dict.keys():
        # Extract activities for the current sector
        sector_activities = scores_dict[sector]['activities']
        
        # Calculate LCA scores using the specified method
        lca_scores = compare_activities_multiple_methods(
            activities_list=sector_activities,
            methods=method_dict,
            identifier=sector,
            mode='absolute'
        )
        
        # Apply the small_inputs_to_other_column function with the cutoff value
        lca_scores = small_inputs_to_other_column(lca_scores, cutoff=0.02)
        
        # Save the LCA scores to the scores_dict
        scores_dict[sector]['lca_scores'] = lca_scores

    return scores_dict

In [15]:
scores_dict = sector_lca_scores(dictionary_one, method_dict)

Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'steel production, electric, '
Omitting activity name common prefix: 'steel production, electric, '


In [27]:
scores_dict_cutoff = sector_lca_scores(dictionary_one, method_dict)

Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'steel production, electric, '
Omitting activity name common prefix: 'steel production, electric, '


In [16]:
import pandas as pd
index_pos=dopo.sector_lca_scores_to_excel_and_column_positions(scores_dict, 'test_dopo_unnamed_1.xlsx')

In [28]:
dopo.sector_lca_scores_to_excel_and_column_positions(scores_dict, 'test_dopo_unnamed_2.xlsx')

{'Cement_global_warming_potential_(gwp100)': {'total': 7,
  'rank': 8,
  'mean': 9,
  '2std_abv': 10,
  '2std_blw': 11,
  'q1': 12,
  'q3': 13,
  'method': 5,
  'method unit': 6,
  'first_input': 14},
 'Cement_cumulative_energy_demand_-_non-renewable_energy_resources': {'total': 7,
  'rank': 8,
  'mean': 9,
  '2std_abv': 10,
  '2std_blw': 11,
  'q1': 12,
  'q3': 13,
  'method': 5,
  'method unit': 6,
  'first_input': 14},
 'Steel_global_warming_potential_(gwp100)': {'total': 7,
  'rank': 8,
  'mean': 9,
  '2std_abv': 10,
  '2std_blw': 11,
  'q1': 12,
  'q3': 13,
  'method': 5,
  'method unit': 6,
  'first_input': 14},
 'Steel_cumulative_energy_demand_-_non-renewable_energy_resources': {'total': 7,
  'rank': 8,
  'mean': 9,
  '2std_abv': 10,
  '2std_blw': 11,
  'q1': 12,
  'q3': 13,
  'method': 5,
  'method unit': 6,
  'first_input': 14}}

In [ ]:
import dopo.plots_in_xcl


current_row=dopo.plots_in_xcl.dot_plots_xcl('test_dopo_3.xlsx', index_pos) #update


In [ ]:
dopo.plots_in_xcl.stacked_bars_xcl('test_dopo_3.xlsx', index_pos, current_row)

comparison of databases

In [ ]:
dopo.relative_changes_db.relative_changes_db(ecoinvent_dict, premise_dict, 'part2_test_dopo_2')

AttributeError: 'function' object has no attribute 'relative_changes_db'

In [ ]:
dopo.plots_in_xcl.barchart_compare_db_xcl('part2_test_dopo_1')